In [43]:
import sys
import numpy as np
import copy
from functools import lru_cache,wraps
from operator import itemgetter,countOf
import collections

def read_file(file_name):
    with open(file_name) as input_file:
        list_of_lines=input_file.readlines()
    return list_of_lines[0].rstrip("\n")

def MakeDict():
    D={}
    D['0']='0000'
    D['1']='0001'
    D['2']='0010'
    D['3']='0011'
    D['4']='0100'
    D['5']='0101'
    D['6']='0110'
    D['7']='0111'
    D['8']='1000'
    D['9']='1001'
    D['A']='1010'
    D['B']='1011'
    D['C']='1100'
    D['D']='1101'
    D['E']='1110'
    D['F']='1111'
    return D

def MakeBinary(line,Dict):
    Packet=''
    for char in line:
        Packet += Dict[char]
    return Packet

def Read_Literal(Packet):
    print('Literal {}'.format(Packet))
    if Packet[0]=='1':
        #print(Packet[1:5])
        Packet=Read_Literal(Packet[5:])
    else:
        #print('Suite {}'.format(Packet[5:]))
        Packet=Packet[5:]
    return Packet
    
def Analyze_Packet(Packet):
    Version=0
    print('Analyze {}'.format(Packet))
    if Packet and '1' in Packet:
        Version=int(Packet[0:3],2)
        Type=int(Packet[3:6],2)
        print('Version {} Type {}'.format(Version,Type))
        if Type==4:
            Packet=Read_Literal(Packet[6:])
            print('Suite {}'.format(Packet))
            Version+=Analyze_Packet(Packet)
        elif Packet[6]=='0':
            print('Operator type 0')
            number=int(Packet[7:22],2)
            print(number)
            print(Packet[22:22+number])
            print(Packet[22+number:])
            Version+=Analyze_Packet(Packet[22:22+number])+Analyze_Packet(Packet[22+number:])
        elif Packet[6]=='1':
            print('Operator type 0')
            number=int(Packet[7:18],2)
            Version+=Analyze_Packet(Packet[18:])
        else:
            print('problem')
            
    return Version

if __name__ == '__main__':
    Line = read_file('input')
    Line='D2FE28'
    Line = '38006F45291200'
    Line='EE00D40C823060'
    Line='8A004A801A8002F478'
    Line='620080001611562C8802118E34'
    Line='C0015000016115A2E0802F182340'
    Line='A0016C880162017C3686B18A3D4780'
    Line = read_file('input')
    Dict=MakeDict()
    Packet=MakeBinary(Line,Dict)
    print(Packet)
    
    Version=Analyze_Packet(Packet)
    print(Version)

0110001000001101011110011000000000101111011000000000100110001000000000111011000100001110001000001100001111000001000000000111101000101110110001001100100001000001001101101111000001100000000010111100101110001010110100000000011001101110001000000000110011000111110110001001110100001100010001000000000111111000011100010000010011100000100101001111111010101000001010110000011100100110011000010011110001101011011010010010010000000000111000010100101000110000010110000000001011010001000100100010001110011000000000100001001001011111101101101001111111110000000000010101000010010101101110011101010010101101110011101110010110110110011110000010000000000101001100000001011101100010001000000000011000101000000000010010111000000000011010111000000000010110001000000000011110110000000100000110000010100000000001010001100000000001111000100000000001010010100000000001010000000000001010100001000110000000000101101000000000100000000000111000000000011110001000000000011000010000000001000110000001000000000011000001101000000000

In [57]:
import sys
import numpy as np
import copy
from functools import lru_cache,wraps
from operator import itemgetter,countOf
import collections
import math

def read_file(file_name):
    with open(file_name) as input_file:
        list_of_lines=input_file.readlines()
    return list_of_lines[0].rstrip("\n")

def MakeDict():
    D={}
    D['0']='0000'
    D['1']='0001'
    D['2']='0010'
    D['3']='0011'
    D['4']='0100'
    D['5']='0101'
    D['6']='0110'
    D['7']='0111'
    D['8']='1000'
    D['9']='1001'
    D['A']='1010'
    D['B']='1011'
    D['C']='1100'
    D['D']='1101'
    D['E']='1110'
    D['F']='1111'
    return D

def MakeBinary(line,Dict):
    Packet=''
    for char in line:
        Packet += Dict[char]
    return Packet

def Read_Literal(Packet):
    if Packet[0]=='1':
        S1=Packet[1:5]
        S2,Packet=Read_Literal(Packet[5:])
        return S1+S2,Packet
    else:
        S1=Packet[1:5]
        return S1,Packet[5:]
    
def Analyze_Packet(Packet):
    Value=0
    print('Analyze {}'.format(Packet))
    
    if Packet and '1' in Packet:
        Version=int(Packet[0:3],2)
        Type=int(Packet[3:6],2)
        print('Version {} Type {}'.format(Version,Type))
        if Type==4:
            String,Packet=Read_Literal(Packet[6:])
            Value=int(String,2)
            print('Literal value {}'.format(Value))
            print('Suite {}'.format(Packet))
        else:
            values=[]
            if Packet[6]=='0':
                print('Operator type 0')
                number=int(Packet[7:22],2)
                Packet_To_Treat=Packet[22:22+number]
                Packet=Packet[22+number:]
                while Packet_To_Treat:
                    sub_value,Packet_To_Treat=Analyze_Packet(Packet_To_Treat)
                    values.append(sub_value)
            else:
                print('Operator type 1')
                number=int(Packet[7:18],2)
                Packet=Packet[18:]
                while number>0:
                    sub_value,Packet=Analyze_Packet(Packet)
                    values.append(sub_value)
                    number-=1
                    
            if Type==0:
                Value=sum(values)                    
            elif Type==1:
                Value=math.prod(values)
            elif Type==2:
                Value=min(values)
            elif Type==3:
                Value=max(values)
            elif Type==5:
                Value=1 if values[0]>values[1] else 0
            elif Type==6:
                Value=1 if values[0]<values[1] else 0
            elif Type==7:
                Value=1 if values[0]==values[1] else 0
    else:
        return 0,''
    return Value,Packet

if __name__ == '__main__':
    Line = read_file('input')
    Line='C200B40A82'
    Line ='04005AC33890'
    Line='880086C3E88112'
    Line='CE00C43D881120'
    Line='D8005AC2A8F0'
    Line='F600BC2D8F'
    Line='9C005AC2F8F0'
    Line='9C0141080250320F1802104A08'
    Line = read_file('input')
    Dict=MakeDict()
    Packet=MakeBinary(Line,Dict)
    print(Packet)
    
    Value,Packer=Analyze_Packet(Packet)
    print(Value)

0110001000001101011110011000000000101111011000000000100110001000000000111011000100001110001000001100001111000001000000000111101000101110110001001100100001000001001101101111000001100000000010111100101110001010110100000000011001101110001000000000110011000111110110001001110100001100010001000000000111111000011100010000010011100000100101001111111010101000001010110000011100100110011000010011110001101011011010010010010000000000111000010100101000110000010110000000001011010001000100100010001110011000000000100001001001011111101101101001111111110000000000010101000010010101101110011101010010101101110011101110010110110110011110000010000000000101001100000001011101100010001000000000011000101000000000010010111000000000011010111000000000010110001000000000011110110000000100000110000010100000000001010001100000000001111000100000000001010010100000000001010000000000001010100001000110000000000101101000000000100000000000111000000000011110001000000000011000010000000001000110000001000000000011000001101000000000